In [30]:
import math
import copy
fread = open("airplane.txt", "r") #netdata.txt
G_v_e = fread.readline()
G_v_e = G_v_e.split('\t') #change to \t if tab spaced
V = int(G_v_e[0])
E = int(G_v_e[1].strip())
#print(E)
V_list = list(range(0,V,1))
lines = fread.readlines()
A_dict = {}
G_mat = [ [ 0 for i in range(4) ] for j in range(E) ]
g = 0
for line in lines:
    t_h = line.split('\t') #change to \t if tab spaced
    #print(t_h)
    G_mat[g][0] = int(t_h[0])
    G_mat[g][1] = int(t_h[1])
    G_mat[g][2] = int(t_h[2])
    G_mat[g][3] = float(t_h[3].replace("\n", ""))
    #print(G_mat[g][3])
    g += 1
for i in range(E):
    tail = G_mat[i][0]
    head = G_mat[i][1]
    if tail in A_dict.keys():
        A_dict[tail].append(head)        
    else:
        A_dict[tail] = [head]

capacity = [[0 for i in range(V)] for j in range(V)]
for i in range(E):
    tail = G_mat[i][0]
    head = G_mat[i][1]
    #print(i)
    #print(tail, head)
    capacity[tail][head] = G_mat[i][3]

print(G_mat, "\n")
print(A_dict)   
print(capacity)
fread.close()

[[0, 1, 0, 150.0], [0, 2, 0, 120.0], [0, 3, 0, 140.0], [0, 4, 0, 75.0], [0, 5, 0, 25.0], [0, 6, 0, 150.0], [0, 7, 0, 20.0], [0, 8, 0, 50.0], [0, 9, 0, 45.0], [0, 10, 0, 20.0], [1, 11, -4100, 10000.0], [1, 12, 0, 10000.0], [2, 11, -8310, 10000.0], [2, 13, 0, 10000.0], [3, 11, -5200, 10000.0], [3, 14, 0, 10000.0], [4, 11, -5900, 10000.0], [4, 15, 0, 10000.0], [5, 12, -5900, 10000.0], [5, 13, 0, 10000.0], [6, 12, -3500, 10000.0], [6, 14, 0, 10000.0], [7, 12, -7100, 10000.0], [7, 15, 0, 10000.0], [8, 13, -4500, 10000.0], [8, 14, 0, 10000.0], [9, 13, -5400, 10000.0], [9, 15, 0, 10000.0], [10, 14, -2000, 10000.0], [10, 15, 0, 10000.0], [12, 16, 0, 150.0], [13, 16, 0, 145.0], [14, 16, 0, 340.0], [15, 16, 0, 160.0], [11, 12, 0, 180.0], [12, 13, 0, 180.0], [13, 14, 0, 180.0], [14, 15, 0, 180.0]] 

{0: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 1: [11, 12], 2: [11, 13], 3: [11, 14], 4: [11, 15], 5: [12, 13], 6: [12, 14], 7: [12, 15], 8: [13, 14], 9: [13, 15], 10: [14, 15], 12: [16, 13], 13: [16, 14], 14: 

In [31]:
def cap_Dijkstra(A_dict, cost, V, s, t):
    parent = {}
    for i in range(V):
        parent[i] = i
    dist = [math.inf]*V
    dist[s] = 0
    fixed = [s]
    chosen = s
    while (t not in fixed):
        if (chosen not in A_dict.keys()):
            print("cant use DJ here")
            return dist[t], parent
        chosen_range = A_dict[chosen]
        k = -1
        for i in range(len(chosen_range)):
            j = chosen_range[i]
            if (j in fixed):
                continue
            if (dist[j] >= dist[chosen] + cost[chosen][j]):
                dist[j] = dist[chosen] + cost[chosen][j]
                parent[j] = chosen
        min_d = 1e8
        min_v = -1
        for v in range(V):
            if ((dist[v] <= min_d) and (v not in fixed) and (dist[v] != math.inf) and (v in A_dict.keys() or v == t)):
                min_d = dist[v]
                min_v = v
        k = min_v
        if (k == -1 or k == math.inf):
            break
        else:
            fixed.append(k)
            chosen = k
    return dist[t], parent


In [32]:
def create_residual_network(G_mat, A_dict, capacity):
    R = copy.deepcopy(G_mat)
    for i in A_dict.keys():
        for j in A_dict[i]:
            if (j in A_dict.keys()):
                if (i in A_dict[j]):
                    continue
            R.append([j, i, 0, 0])
            #capacity[j][i] = 0
    return R


def find_s_t_path(R, s, end, V, capacity): # Use R for nodes & arc connects and 'capacity' for residual capacity
    mod_cost = [[math.inf for i in range(V)] for j in range(V)]
    R_dict = {}
    
    #print('capacity')
    #for i in range(V):
    #    print(capacity[i])
        
    # Develop modified cost & dictionary
    for e in range(len(R)):
        t = R[e][0]
        h = R[e][1]
        if (capacity[t][h] != 0):
            mod_cost[t][h] = 1
            if (t in R_dict.keys()):
                if (h not in R_dict[t]):
                    R_dict[t].append(h)
            else:
                R_dict[t] = [h]
        else:
            if (t in R_dict.keys()):
                if (h in R_dict[t]):
                    R_dict[t].remove(h)
        
    """
    mod_cost[3][7] = 0
    mod_cost[8][12] = 0
    mod_cost[17][18] = 0
    mod_cost[7][3] = 0
    mod_cost[12][8] = 0
    mod_cost[18][17] = 0
    """
    
    #Print values
    #print('modified cost')
    #for i in range(V):
    #    print(mod_cost[i])
    #print(R_dict)
    
    # Find shortest s-t path
    min_cost, parent = cap_Dijkstra(R_dict, mod_cost, V, s, end)
    #print(min_cost)
    #print(parent)
    path = []
    if (min_cost == math.inf):
        return path
    path = [end]
    curr = end
    while (curr != s and curr != parent[curr]):
        curr = parent[curr]
        path.insert(0, curr)
        #print(path)   
    return path

def retrieve_flow(G_mat, R, capacity):
    flow = []
    for i in range(E):
        t = G_mat[i][0]
        h = G_mat[i][1]
        old = G_mat[i][3]
        new = capacity[t][h]
        if (old > new):
            flow.append(old - new)
        else:
            flow.append(0)            
    return flow

def FF(G_mat, A_dict, capacity, V, s, end):
    R = create_residual_network(G_mat, A_dict, capacity)
    #print(R)
    
    #extra    
    """
    capacity[3][7] = 19
    capacity[8][12] = 29
    capacity[17][18] = 22
    capacity[7][3] = 0
    capacity[12][8] = 0
    capacity[18][17] = 0
    """
    

    
    
    
    flow_value = 0
    path = find_s_t_path(R, s, end, V, capacity)
    #print(path)
    while (len(path) != 0):
        min_cap = math.inf
        for n in range(len(path)-1):
            t = path[n]
            h = path[n+1]
            if (capacity[t][h] < min_cap):
                min_cap = capacity[t][h]
        #print('here')
        #print(min_cap)
        flow_value = flow_value + min_cap
        for n in range(len(path)-1):
            t = path[n]
            h = path[n+1]
            capacity[t][h] = capacity[t][h] - min_cap
            capacity[h][t] = capacity[h][t] + min_cap
        path = find_s_t_path(R, s, end, V, capacity)
        #print(path)
    flow = retrieve_flow(G_mat, R, capacity)
    #print('flow')
    #print(flow)
    return flow, flow_value
   

In [33]:
s = 0
t = 16
capacity = [[0 for i in range(V)] for j in range(V)]
for i in range(E):
    tail = G_mat[i][0]
    head = G_mat[i][1]
    capacity[tail][head] = G_mat[i][3]
#capacity = [[0 for i in range(V)] for j in range(V)]
    
"""
capacity[3][7] = 16
capacity[8][12] = 24
capacity[17][18] = 15
capacity[7][3] = 3
capacity[12][8] = 5
capacity[18][17] = 7
"""

flow, flow_value = FF(G_mat, A_dict, capacity, V, s, t) 
print('Max flow: ', flow_value)
#printing flow through arcs
print('Flow through arcs-')
for i in range(E):
    print('(', G_mat[i][0],',' , G_mat[i][1], ')', ':', flow[i])

cant use DJ here
Max flow:  795.0
Flow through arcs-
( 0 , 1 ) : 150.0
( 0 , 2 ) : 120.0
( 0 , 3 ) : 140.0
( 0 , 4 ) : 75.0
( 0 , 5 ) : 25.0
( 0 , 6 ) : 150.0
( 0 , 7 ) : 20.0
( 0 , 8 ) : 50.0
( 0 , 9 ) : 45.0
( 0 , 10 ) : 20.0
( 1 , 11 ) : 0
( 1 , 12 ) : 150.0
( 2 , 11 ) : 0
( 2 , 13 ) : 120.0
( 3 , 11 ) : 0
( 3 , 14 ) : 140.0
( 4 , 11 ) : 0
( 4 , 15 ) : 75.0
( 5 , 12 ) : 0
( 5 , 13 ) : 25.0
( 6 , 12 ) : 0
( 6 , 14 ) : 150.0
( 7 , 12 ) : 0
( 7 , 15 ) : 20.0
( 8 , 13 ) : 0
( 8 , 14 ) : 50.0
( 9 , 13 ) : 0
( 9 , 15 ) : 45.0
( 10 , 14 ) : 0
( 10 , 15 ) : 20.0
( 12 , 16 ) : 150.0
( 13 , 16 ) : 145.0
( 14 , 16 ) : 340.0
( 15 , 16 ) : 160.0
( 11 , 12 ) : 0
( 12 , 13 ) : 0
( 13 , 14 ) : 0
( 14 , 15 ) : 0
